#### 코드 파트

In [8]:
import requests
import re
import pandas as pd
import urllib.request as req
from bs4 import BeautifulSoup

In [9]:
def blog_crawling(start_day, end_day, keyword):
    URLlist = []
    first_url = ''
    for j in range(100):
        param = {
            "where" : "post",
            "query" : keyword,
            "date_from" : start_day,
            "date_to" : end_day,
            "date_option" : "8",
            "start" : str(10*j+1) 
        }
        response = requests.get(url,params=param, headers=hrd)
        print("--------",response.url,"-----------")
        f = open("C:/Users/Joobe/Downloads/NaverBlog_{0:}.txt".format(keyword), "a", encoding="utf-8")
        soup = BeautifulSoup(response.text, "html.parser")
        for links in soup.select("li.sh_blog_top > dl"):
            try:
                title_html = links.select("dt>a")
                content_html = links.select("dd.sh_blog_passage")
                author_url_html = links.select("dd.txt_block a")
                title = title_html[0].get('title')
                content = content_html[0].text
                author = author_url_html[0].text
                URL = author_url_html[1].get('href')
                URL = get_final_url(URL)
                if URL == first_url:
                    print("중복. 다음 기간으로 넘어갑니다.")
                    return
                if URL =="none": continue
                URLlist.append(URL)
                res = req.urlopen(URL)
                soup = BeautifulSoup(res, 'html.parser')
                cont = soup.find("div",{"class":"se-main-container"})
                content = cont.get_text()
            except:
                try: 
                    cont = soup.find("div",{"id":"postViewArea"})
                    content = cont.get_text()
                except:
                    try:
                        content = ' '.join([x.get_text() for x in soup.findAll("p",{"class":"se_textarea"})])
                    except:
                        print("\n\n" + URL + "예외처리발생\n\n")
                        return
            finally:
                filtered_content = re.sub("[\n\u200b]"," ",content)
                filtered_content = re.sub("\s+", " ", filtered_content)
            try: 
                f.write(filtered_content+"\n")
            except:
                print("파일에작성되지않음")
            #print(keyword, response.url)
            #blog_crawling(keyword, response.url)
            #print(response.text)
            #print(type(response.url))
        if URLlist:
            first_url = URLlist[0]
        else:
            return
        URLlist.clear()
        f.close()
        #if j==2: break

In [10]:
url = "http://search.naver.com/search.naver"
hrd = {"User-Agent":"Mozilla/5.0","referer":"http://naver.com"}
start_days = pd.date_range(start='20160101', end='20190731', freq="MS")
end_days = pd.date_range(start="20160101",end="20190731",freq="M")

# 2016.01 ~ 2019.07 동안 총 43달 == len(start_days)

def NaverBlogCrawl(keyword):
    for i in range(len(start_days)):
        print("**새로운 기간 시작**")
        start_day = start_days[i].strftime("%Y%m%d")
        end_day = end_days[i].strftime("%Y%m%d")
        blog_crawling(start_day, end_day, keyword)

#### 실행 파트

In [5]:
NaverBlogCrawl("장호항") # NaverBlogCrawl("검색어")

**새로운 기간 시작**
-------- https://search.naver.com/search.naver?where=post&query=SOFEX&date_from=20160101&date_to=20160131&date_option=8&start=1 -----------
